# Here we use bagging regressor, extra tree regressor, voting regressor and random forest regressor on top of Household consumption data

### Loading the needed libraries


In [3]:
### Loading the needed libraries
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline

# Loading Data

In [5]:
url = 'https://raw.githubusercontent.com/LGLV/ML_EnergyConsumption/master/hep.csv'

In [6]:
data = pd.read_csv(url)
data

,Unnamed: 0,Global_active_power,Global_reactive_power,Voltage,Sub_metering_1,Sub_metering_2,Sub_metering_3,converted_Date_week,Time_h,Time_min,target
0,0,4.216,0.418,234.84,0,1,17.0,50,17,24,18.0
1,1,5.360,0.436,233.63,0,1,16.0,50,17,25,17.0
2,2,5.374,0.498,233.29,0,2,17.0,50,17,26,19.0
3,3,5.388,0.502,233.74,0,1,17.0,50,17,27,18.0
4,4,3.666,0.528,235.68,0,1,17.0,50,17,28,18.0
...,...,...,...,...,...,...,...,...,...,...,...
48871,49996,3.754,0.542,232.07,37,2,0.0,3,10,40,39.0
48872,49997,3.750,0.540,231.90,37,1,0.0,3,10,41,38.0
48873,49998,1.876,0.568,233.95,7,1,0.0,3,10,42,8.0
48874,49999,1.578,0.490,234.52,1,1,0.0,3,10,43,2.0


In [13]:
data.describe().T

,count,mean,std,min,25%,50%,75%,max
Unnamed: 0,48876.0,25001.729581,14433.362556,0.000,12468.750,25000.500,37494.25,50000.000
Global_active_power,48876.0,1.601715,1.249243,0.194,0.392,1.412,2.42,8.698
Global_reactive_power,48876.0,0.135405,0.118618,0.000,0.000,0.126,0.20,0.862
Voltage,48876.0,241.121522,3.535250,226.320,238.740,241.360,243.62,251.700
Sub_metering_1,48876.0,0.647414,4.568763,0.000,0.000,0.000,0.00,53.000
Sub_metering_2,48876.0,1.300106,5.389895,0.000,0.000,0.000,1.00,52.000
Sub_metering_3,48876.0,6.994251,8.581001,0.000,0.000,0.000,17.00,20.000
converted_Date_week,48876.0,31.548920,21.135210,1.000,5.000,40.000,51.00,52.000
Time_h,48876.0,11.438702,6.976467,0.000,5.000,11.000,18.00,23.000
Time_min,48876.0,29.567047,17.319869,0.000,15.000,30.000,45.00,59.000


# Splitting in X and y variables

In [74]:
to_consider = ['Global_active_power', 'Global_reactive_power', 'Voltage', 'Sub_metering_1', 'Sub_metering_2', 'Sub_metering_3', 'converted_Date_week', 'Time_h', 'Time_min']
X = data[to_consider]
X

,Global_active_power,Global_reactive_power,Voltage,Sub_metering_1,Sub_metering_2,Sub_metering_3,converted_Date_week,Time_h,Time_min
0,4.216,0.418,234.84,0,1,17.0,50,17,24
1,5.360,0.436,233.63,0,1,16.0,50,17,25
2,5.374,0.498,233.29,0,2,17.0,50,17,26
3,5.388,0.502,233.74,0,1,17.0,50,17,27
4,3.666,0.528,235.68,0,1,17.0,50,17,28
...,...,...,...,...,...,...,...,...,...
48871,3.754,0.542,232.07,37,2,0.0,3,10,40
48872,3.750,0.540,231.90,37,1,0.0,3,10,41
48873,1.876,0.568,233.95,7,1,0.0,3,10,42
48874,1.578,0.490,234.52,1,1,0.0,3,10,43


In [75]:
y = data.target
y

0        18.0
1        17.0
2        19.0
3        18.0
4        18.0
         ... 
48871    39.0
48872    38.0
48873     8.0
48874     2.0
48875     8.0
Name: target, Length: 48876, dtype: float64

# Spliting data in train and test sets

In [76]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)
print('The shape of X train is:', X_train.shape)
print('The shape of y train is:', y_train.shape)
print('The shape of X test is:', X_test.shape)
print('The shape of y test is:', y_test.shape)

The shape of X train is: (32746, 9)
The shape of y train is: (32746,)
The shape of X test is: (16130, 9)
The shape of y test is: (16130,)


# Feature scaling

In [77]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

In [78]:
X_train_esc = scaler.fit_transform(X_train)

In [79]:
X_test_esc = scaler.transform(X_test)

# Training 4 regression models

In [80]:
from sklearn.ensemble import BaggingRegressor, ExtraTreesRegressor, VotingRegressor, RandomForestRegressor

In [81]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.model_selection import cross_val_score, KFold

In [93]:
# lets create a Predictive Models

model_accuracy = pd.DataFrame(columns=['Model'])
models = {
    "BAG" : BaggingRegressor(),
    "ETR" : ExtraTreesRegressor(),
    "RFR" : RandomForestRegressor(),
    "VR" : VotingRegressor([('BAG', BaggingRegressor()), ('ETR', ExtraTreesRegressor()), ('RFR', RandomForestRegressor())])
    }

for abrev, model in models.items():
    
    model.fit(X_train_esc, y_train)
    
    score = model.score(X_train_esc, y_train) # Training score
        
    y_pred = model.predict(X_test_esc)
    
    pred_accurasy_MAE = mean_absolute_error(y_test, y_pred)
    pred_accurasy_MSE = mean_squared_error(y_test, y_pred)
    pred_accurasy_R2 = r2_score(y_test, y_pred) # Accurasy score
    
    
    model_accuracy = model_accuracy.append({
        'Model': abrev, 
        'r2': pred_accurasy_R2,
        'MAE': pred_accurasy_MAE,
        'MSE': pred_accurasy_MSE,
        'Training_scores': score,
    },ignore_index=True)
    
    print('\n', abrev + ' scores')
    print(score)
    print('MAE:', pred_accurasy_MAE)
    print('MSE:', pred_accurasy_MSE)
    print('r2:', pred_accurasy_R2)
    # model_accuracy = model_accuracy.append({'Model': abrev, 'Accuracy': score, 'Train_acc': score}, ignore_index=True)        


 BAG scores
0.9999306027529549
MAE: 0.023434593924364554
MSE: 0.08224922504649722
r2: 0.9993686412420352

 ETR scores
1.0
MAE: 0.010596404215747056
MSE: 0.03931840049597024
r2: 0.9996981854055371

 RFR scores
0.9999633222324672
MAE: 0.02096280223186609
MSE: 0.07246192808431494
r2: 0.9994437701645312

 VR scores
0.9999806105516897
MAE: 0.017296548873734222
MSE: 0.0585313411861955
r2: 0.9995507036710384


In [94]:
model_accuracy.sort_values(ascending=False, by = 'Training_scores')

,Model,MAE,MSE,Training_scores,r2
1,ETR,0.010596,0.039318,1.000000,0.999698
3,VR,0.017297,0.058531,0.999981,0.999551
2,RFR,0.020963,0.072462,0.999963,0.999444
0,BAG,0.023435,0.082249,0.999931,0.999369


# Preliminary results

* According to the training score Extra tree Regression model overfits the model.
* Thus, we might choose any other model, nevertheless all are prone to overfit.
* It is mandatory to work better with the raw data.
